In [1]:
import numpy as np

In [2]:
class _LinearRegression():
    def __init__(self, alpha=0.03, n_iter=1500, method="gradient descent"):
        self.alpha = alpha
        self.n_iter = n_iter
        self.method = method
        self.params = None

    def _gradient_descent(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        
        X_train = np.hstack((np.ones((n_samples, 1)), (X_train - np.mean(X_train, 0)) / np.std(X_train, 0)))
        y_train = y_train[:, np.newaxis]
        self.params = np.zeros((n_features + 1, 1))

        for i in range(self.n_iter):
            self.params -= (self.alpha/n_samples) * X_train.T @ (X_train @ self.params - y_train)

        return self
    
    def _sgd(self, X_train, y_train):
        pass
    
    def _fit(self, X_train, y_train):
        if self.method == "gradient descent":
            return self._gradient_descent(X_train, y_train)
        
        elif self.method == "sgd":
            return self._sgd(X_train, y_train)

    def _predict(self, X_test):
        n_samples = np.size(X_test, 0)
        y = np.hstack((np.ones((n_samples, 1)), (X_test - np.mean(X_test, 0)) / np.std(X_test, 0))) @ self.params
        return y

In [3]:
from sklearn.datasets import load_boston

boston = load_boston()
data = boston.data
target = boston.target

data.shape       

(506, 13)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

print(f'Train data shape: {X_train.shape}')
print(f'Test data shape: {X_test.shape}')

Train data shape: (354, 13)
Test data shape: (152, 13)


In [5]:
from sklearn.metrics import mean_squared_error

In [6]:
%%time

_lr_custom = _LinearRegression()
_lr_custom._fit(X_train, y_train)
y_pred_custom = _lr_custom._predict(X_test)
mse_custom = mean_squared_error(y_test, y_pred_custom)

print(f"Mean squared error for custom model: {mse_custom}")

Mean squared error for custom model: 18.306198830723208
Wall time: 207 ms


In [7]:
from sklearn.preprocessing import StandardScaler
 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
%%time

from sklearn.linear_model import LinearRegression

lr_sklearn = LinearRegression()
lr_sklearn.fit(X_train, y_train)
y_pred_sklearn = lr_sklearn.predict(X_test)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)

print(f"Mean squared error for sklearn model: {mse_sklearn}")

Mean squared error for sklearn model: 18.337323968639005
Wall time: 513 ms


#### Explanation of matrix operations

In [58]:
x = np.array([[1, 2 , 3, 4],
              [2, 3, 2, 5],
              [1, 5, 4, 2],
              [2, 3, 4, 5]])

In [59]:
w = np.array([2, 1, 3, 4])

In [60]:
x @ w # Returns prediction for each object

array([29, 33, 27, 39])

In [62]:
x.T @ (x @ w) # Sum of multiplication of prediction to each column

array([200, 409, 417, 530])